In [1]:
! pip install numpy==1.26.4

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ritvik1909/document-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/document-classification-dataset


In [3]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from datasets import Dataset, Features, Sequence, ClassLabel, Value, Array2D

In [4]:
labels = [label for label in os.listdir(path)]
idx2label = {v: k for v, k in enumerate(labels)}
label2idx = {k: v for v, k in enumerate(labels)}
label2idx

{'resume': 0, 'email': 1, 'scientific_publication': 2}

In [5]:
class_folders = os.listdir(path)
class_to_label = {class_folder: index for index, class_folder in enumerate(class_folders)}

images = []
labels = []

for class_folder in class_folders:
    class_path = os.path.join(path, class_folder)
    for image_filename in os.listdir(class_path):
        image_path = os.path.join(class_path, image_filename)
        images.append(image_path)
        labels.append(class_to_label[class_folder])

d = {label: labels.count(label) for label in labels}
d

{0: 55, 1: 55, 2: 55}

In [6]:
! python -m pip install paddlepaddle-gpu==3.0.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu118/

Looking in indexes: https://www.paddlepaddle.org.cn/packages/stable/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 544.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.9/699.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
! pip install -qU paddleocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [8]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(
    use_doc_orientation_classify=True,
    use_doc_unwarping=True,
    use_textline_orientation=True
)

Connecting to https://paddle-model-ecology.bj.bcebos.com/paddlex/PaddleX3.0/fonts/PingFang-SC-Regular.ttf ...
[==================================================] 100.00%
Connecting to https://paddle-model-ecology.bj.bcebos.com/paddlex/PaddleX3.0/fonts/simfang.ttf ...
[==================================================] 100.00%


/usr/local/lib/python3.11/dist-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in /root/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/7.06k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/352k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

inference.pdiparams:   0%|          | 0.00/84.4M [00:00<?, ?B/s]

inference.yml:   0%|          | 0.00/148k [00:00<?, ?B/s]

inference.json:   0%|          | 0.00/325k [00:00<?, ?B/s]

In [9]:
def normalize_box(box, width, height):
    return [
        int(1000 * (box[0] / width)),
        int(1000 * (box[1] / height)),
        int(1000 * (box[2] / width)),
        int(1000 * (box[3] / height)),
    ]

def get_box_from_quad(quad):
    """
    Converts a 4-point quadrilateral into a rectangular bounding box.
    Input: quad - list of 4 points: [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]
    Output: [x_min, y_min, x_max, y_max]
    """
    x_coords = [point[0] for point in quad]
    y_coords = [point[1] for point in quad]
    return [min(x_coords), min(y_coords), max(x_coords), max(y_coords)]

In [10]:
from PIL import Image, ImageDraw, ImageFont

In [11]:
def apply_ocr(examples):
    words = []
    for example in examples:
      image = Image.open(example).convert("L")
      width, height = image.size

      # Run PaddleOCR
      result = ocr.predict(example)

      doc_words = []
      boxes = []

      ocr_data = result[0]

      # Get texts and boxes from the OCR result
      rec_texts = ocr_data['rec_texts']
      rec_boxes = ocr_data['rec_boxes']

      for i, text in enumerate(rec_texts):
          if not text.strip():
              continue

          box = rec_boxes[i]

          # Ensure box has 4 coordinates
          if len(box) == 4:
              box = [int(coord) for coord in box]
              norm_box = normalize_box(box, width, height)
              doc_words.append(text)
              boxes.append(norm_box)
      words.append(" ".join(doc_words))

    return words

In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch

# Preprocess the extracted texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
images[0]

'/kaggle/input/document-classification-dataset/resume/doc_000551.png'

In [14]:
extracted_texts = apply_ocr(images)
extracted_texts = [extracted_texts[idx].replace("\n", ' ') for idx, value in enumerate(extracted_texts)]

In [15]:
from datasets import Dataset

In [16]:
tokenized_texts = tokenizer(extracted_texts, padding=True, truncation=True, return_tensors='pt')

tokenized_texts.input_ids.shape, tokenized_texts.attention_mask.shape

(torch.Size([165, 512]), torch.Size([165, 512]))

In [17]:
# Split data into training and validation sets
input_ids = tokenized_texts.input_ids
attention_mask = tokenized_texts.attention_mask
labels = torch.tensor(labels)


print("Input ids Shape:",input_ids.shape)
print("attention_mask Shape:",attention_mask.shape)
print("labels Shape:",labels.shape)

train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state=42, test_size=0.2, shuffle=True)
train_masks, val_masks, _, _ = train_test_split(attention_mask, input_ids, random_state=42, test_size=0.2, shuffle=True)

print()

print("Train Input ids Shape:",train_inputs.shape)
print("Train attention_mask Shape:",train_masks.shape)
print("Train labels Shape:",train_labels.shape)

print()

print("Test Input ids Shape:",val_inputs.shape)
print("Test attention_mask Shape:",val_masks.shape)
print("Test labels Shape:",val_labels.shape)

print()

print("Train Sample Count[Class-wise]", {label: list(train_labels.numpy()).count(label) for label in list(train_labels.numpy())})
print("Test Sample Count[Class-wise]", {label: list(val_labels.numpy()).count(label) for label in list(val_labels.numpy())})

Input ids Shape: torch.Size([165, 512])
attention_mask Shape: torch.Size([165, 512])
labels Shape: torch.Size([165])

Train Input ids Shape: torch.Size([132, 512])
Train attention_mask Shape: torch.Size([132, 512])
Train labels Shape: torch.Size([132])

Test Input ids Shape: torch.Size([33, 512])
Test attention_mask Shape: torch.Size([33, 512])
Test labels Shape: torch.Size([33])

Train Sample Count[Class-wise] {1: 44, 0: 44, 2: 44}
Test Sample Count[Class-wise] {2: 11, 1: 11, 0: 11}


In [18]:
# Create custom dataset and dataloaders
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

train_dataset = CustomDataset(train_inputs, train_masks, train_labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True)

val_dataset = CustomDataset(val_inputs, val_masks, val_labels)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, drop_last=True)

In [19]:
# Initialize and train BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device::", device)
# device = torch.device("cuda")
model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {val_loss / len(val_loader)}')

# Save the trained model
model.save_pretrained('bert_document_classification_model')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device:: cuda
Epoch 1/5, Loss: 0.5620048493146896
Epoch 2/5, Loss: 0.2452890630811453
Epoch 3/5, Loss: 0.1325191566720605
Epoch 4/5, Loss: 0.07279891800135374
Epoch 5/5, Loss: 0.05864276271313429


In [20]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the trained model
model_path = 'bert_document_classification_model'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval()

# Function to predict document class
def predict_document_class(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

In [22]:
# Put the model in evaluation mode
model.eval()

# Initialize variables to calculate accuracy
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Get model predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        # Update accuracy metrics
        total_correct += torch.sum(predictions == labels).item()
        total_samples += len(labels)

# Calculate accuracy
accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


In [23]:
# Put the model in evaluation mode
model.eval()

# Iterate through the validation data
with torch.no_grad():
    for batch_idx, batch in enumerate(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Get model predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Print the real labels and the input values for the first batch as an example
        if batch_idx == 0:
            print("Real labels for the first validation batch:", labels)
            print("Input IDs (X values) for the first validation batch:", input_ids)
            # You can similarly print attention_mask if needed
            # print("Attention Mask for the first validation batch:", attention_mask)
            break # Just showing the first batch

Real labels for the first validation batch: tensor([2, 2, 2, 1], device='cuda:0')
Input IDs (X values) for the first validation batch: tensor([[  101,  1024, 28194,  ...,  1997, 24098,   102],
        [  101,  1996,  3485,  ...,  2459,  1007,   102],
        [  101,  1769,  4261,  ...,  1012,  1006,   102],
        [  101,  6643,  5910,  ...,     0,     0,     0]], device='cuda:0')


In [24]:
# Decode the input IDs back to text for the first batch
decoded_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]

print("Decoded texts for the first validation batch:")
for i, text in enumerate(decoded_texts):
    print(f"Sample {i+1}: {text}")

Decoded texts for the first validation batch:
Sample 1: : wat2 cancinst 5o : 1243 - 57, 1973 comparison of neoplasms in slx sources of rats william f. mac kenie, l. colone, usaf, vcond f. m. ganer, colone, vc, usa, retiredveer - inary pothology division, armed forces institute of pathology, washington, d. c. 20306 summary - we described 749 tumors from 6 sources of rats raised at 7 laboratorles. the sources were " sprague - dawley " from4commercial suppliers, osborne - mendel, and oregon. highlysigniicant differ - ences in the incidences of endocrine and mammary tumors were found. there were significant variationsin the incidences of odrenal medullary tumors among rats from the same source raised at 2 laboratories. of therats with endocrine tumors, 9 - 15 % had tumors in 2 or more endocrine gland. allbut 1 of the testicular tumors oecurred in oregon rat. thyrold and pitultary tumors were more common in females, adrenal medul - lary and islet cell tumors of the pancreas were more freque

In [26]:
! pip install -qU huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

# Login to Hugging Face Hub
notebook_login()

# Push the model and tokenizer to the hub
repo_id = "visithck/Bert-Based-Docu-classify"
model.push_to_hub(repo_id, token="")
tokenizer.push_to_hub(repo_id, token="")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/visithck/Bert-Based-Docu-classify/commit/725f6cefd2170f47a1dd1aa1909a1811fde6a0ba', commit_message='Upload tokenizer', commit_description='', oid='725f6cefd2170f47a1dd1aa1909a1811fde6a0ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/visithck/Bert-Based-Docu-classify', endpoint='https://huggingface.co', repo_type='model', repo_id='visithck/Bert-Based-Docu-classify'), pr_revision=None, pr_num=None)

In [35]:
output_dir = "./saved_model_and_tokenizer"
os.makedirs(output_dir, exist_ok=True)

# Save model
model.save_pretrained(output_dir)

# Save tokenizer
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to ./saved_model_and_tokenizer


In [36]:
import shutil

# Define the output zip file name
zip_filename = "document-classification-dataset.zip"
output_zip_path = f"./{zip_filename}"

# Create a zip archive of the folder
# The 'base_name' is the name of the archive without the extension
# The 'format' is the archive format ('zip', 'tar', 'gztar', 'bztar', 'xztar')
# The 'root_dir' is the directory to start archiving from (the parent of the folder you want to zip)
# The 'base_dir' is the directory inside root_dir that you want to archive (the folder itself)
shutil.make_archive(output_zip_path.replace('.zip', ''), 'zip', os.path.dirname(path), os.path.basename(path))

print(f"Folder '{os.path.basename(path)}' successfully zipped to '{output_zip_path}'")

# Now you can download the zip file using the files.download() function from google.colab
from google.colab import files
files.download(output_zip_path)

Folder 'document-classification-dataset' successfully zipped to './document-classification-dataset.zip'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>